# CSE 555 - Introduction to pattern recognition
## Problem Set 1: Linear Discriminant Functions and Support Vector Machines




## PART 1

Dataloader to read the MNIST dataset. The dataset is read and is flattened. Hence each 28x28 image is now a vector of length 784.


In [0]:
#Using data loader from previous assignment

import numpy as np

def load_mnist(kind='train'):
    import os
    import gzip

    """Load MNIST data from `path`"""
    labels_path = ('%s-labels-idx1-ubyte.gz' % kind)
    images_path = ('%s-images-idx3-ubyte.gz' % kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 784)

    return images, labels


x_train, y_train = load_mnist(kind='train')
x_test, y_test = load_mnist(kind='t10k')

Preprocessing the data so that the mean of the dataset is zero and the standard deviation is one.

In [0]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

The penalty is chosen as 'l1' since the requirement is to use '1-norm soft margin'.
The regularization parameter (C) is a hyper-parameter. This is tuned using cross-validation.

In [0]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV

parameters = {'C':[0.001, 0.01, 0.1, 1]}
svc = svm.LinearSVC(loss='squared_hinge', penalty='l1', dual=False)
clf = GridSearchCV(svc, parameters)
# clf = svm.LinearSVC(penalty='l1')
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)

The best value for regularization parameter is obtained through cross-validation.

In [16]:
print("Best C from cross validation: ", clf.best_params_)

Best C from cross validation:  {'C': 0.1}


The performance of the model is evaluated through accuracy and confusion matrix as metrics.

In [0]:
from sklearn.metrics import confusion_matrix
print("score = %3.2f" %(clf.score(x_test, y_test)))
print("confusion matrix: \n ", confusion_matrix(y_test, y_pred))

## PART 2

Given:
<br>
features 
\begin{equation}
\left( x_{1} ,\ y_{1}\right) ,...,\left( x_{N} ,\ y_{N}\right) \ where\ y_{1} ,...,y_{N} \ \in \{-1,\ 1\}
\end{equation}
<br>
We need to minimize: 
\begin{equation*}
W^{T} .W+C\sum ^{N}_{i=1} \xi _{i}
\end{equation*}
<br>
i.e., the weighted sum between the squared length of the separating vector and the errors, where $\displaystyle W$ is the separating vector.
<br>
<br>
s.t 
\begin{equation*}
y_{i} .\left( W^{T} .x_{i}\right) \geq 1-\xi _{i}
\end{equation*}
<br>
and
\begin{gather*}
\xi _{i} \geq 0
\end{gather*}
<br>
for $\displaystyle i=1,...,N$.

### Objective: To form the Lagrangian dual problem

Using slide 4/27:
The primal margin is :
 
\begin{gather*}
\gamma \ =\ \frac{1}{\sqrt{W^{T} .W+C\sum ^{N}_{i=1} \xi _{i}}}\\
\end{gather*}

Using slide 5/27:
Lagrange function:

\begin{equation*}
L=\overrightarrow{W}^{T}\overrightarrow{W} +C\sum ^{N}_{i=1} \xi _{i} +\sum ^{N}_{i=1} α_{i}\left( 1−y_{i} \cdot W^{T} x_{i} -\xi _{i}\right) -\sum ^{N}_{i=1} \beta _{i} \xi _{i}
\end{equation*}
with
<br>
Lagrange multipliers: $\displaystyle \alpha _{i} \geq 0\ ,\ \beta _{i} \geq 0$,
<br>
Inequality constraits: $\displaystyle 1−𝑦_{i}\left( W^{T} x_{i}\right) −\xi _{i} \leq 0$ and $\displaystyle \xi _{i} \geq 0$
<br>
for all $\displaystyle i=1,...,N$
#### Claim: <br>
$\underbrace{\underset{\alpha,\beta}{max} \underset{W,\xi}{min}{L}}_\text{dual solution} \leq \underbrace{\underset{W,\xi}{min} \underset{\alpha,\beta}{max}{L}}_\text{primal solution}$

\begin{equation*}
L=\overrightarrow{W}^{T}\overrightarrow{W} +C\sum ^{N}_{i=1} \xi _{i} +\sum ^{N}_{i=1} α_{i}\left( 1−y_{i} \cdot W^{T} x_{i} -\xi _{i}\right) -\sum ^{N}_{i=1} \beta _{i} \xi _{i}
\end{equation*}


\begin{equation*}
\\
  {\partial_{{W}}}{L} = 2W-\sum ^{N}_{i=1} \alpha_i y_i \overrightarrow{x_i} \stackrel{\text{set}}{=} 0 \implies 2W = {\sum ^{N}_{i=1} \alpha_i y_i \overrightarrow{x_i}}
  \\
  {\partial_{{\xi}}}{L} = C - \alpha_i - \beta_i = 0
  \\ 
  \implies 0 \leq \alpha_i \leq C \text{ and } \beta_i \geq 0
\end{equation*}

Second order partial derivative \begin{gather*} \partial^2_w\mathcal L=1 \gt 0:
\end{gather*}
<br>
\begin{gather*}
2W=\sum ^{N}_{i=1}\alpha_i y_i \vec x_i\ \text{minimizes L with given } \alpha_i,\forall i \end{gather*}

Substituting \begin{gather*} 2W = {\sum ^{N}_{i=1} \alpha_i y_i \overrightarrow{x_i}}, C=\alpha_i + \beta_i \end{gather*}
into Lagrange function, we get the dual problem of maximizing:

\begin{equation*}
{{L}} = \overrightarrow{W}^{T} \overrightarrow{W} + (\alpha_i + \beta_i) \sum_{i=1}^N\xi_i + \sum_{i=1}^N\alpha_i(1-y_i.\overrightarrow{W}^{T} \overrightarrow{x_i} -\xi_i) - \sum_{i=1}^N \beta_i \xi_i
\\\quad = W^T \overrightarrow{W} + \alpha_i\sum_{i=1}^N\xi_i + \beta_i \sum_{i=1}^N\xi_i +\sum_{i=1}^N \alpha_i - \overrightarrow{W}^{T} \sum_{i=1}^N \alpha_i y_i \overrightarrow{x_i} - \alpha_i \sum_{i=1}^N \xi_i - \beta_i \sum_{i=1}^N \xi_i
\\\quad = W^T \overrightarrow{W} + \alpha_i\sum_{i=1}^N\xi_i + \beta_i \sum_{i=1}^N\xi_i +\sum_{i=1}^N \alpha_i - \overrightarrow{W}^{T} 2\overrightarrow{W} - \alpha_i \sum_{i=1}^N \xi_i - \beta_i \sum_{i=1}^N \xi_i
\end{equation*}

\begin{equation*}
\\=\sum_{i=1}^N \alpha_i-\sum_{i,j=1}^N \overrightarrow{W}^{T}\alpha_i y_i \overrightarrow{x_i}
\end{equation*}
\begin{equation*}
\\=\sum_{i=1}^N \alpha_i-\sum_{i,j=1}^N \alpha_i \alpha_j y_i y_j \overrightarrow{x_i}^{T}\overrightarrow{x_j}
\end{equation*}
<br>
**The dual margin is :**
 
\begin{gather*}
\gamma \ =\ \frac{1}{\sqrt{\alpha_i \alpha_j y_i y_j \overrightarrow{x_i}^{T}\overrightarrow{x_j}}}\\
\end{gather*}

#### Benefits of maximizing the margin
1. Increasing the margin leads to ignoring the points close to the boundary. That is, the samples inside the margins are penalized less.
2. Having a high margin in cases of data with outliers would be better so that the algorithm can generalize.
3. Having a high margin would help to avoid overfitting.

#### Support vectors characteristics:
1. Support vectors are the points that are crucial in building SVM.
2. These are data points that are closer to the hyperplane and influence the position and orientation of the hyperplane.
3. The support vectors are those points for which the Lagrange multiplier is not zero.

#### Benefits of solving the dual problem instead of the primal problem
1. The biggest advantage of solving the dual problem comes when the kernel trick is used. It allows us to classify data that is not linearly separable in the original feature space.
2. In dual problem, it is not necessary to explicitly compute the mapping for each data point as in the case of the primal problem.
3. The dual space has only as many parameters as the number of data points irrespective of the dimensions.
4. Regularizing the sparse support vector associated with the dual hypothesis is sometimes more intuitive than regularizing the vector of regression coefficients.

<!-- ## PART 3
### Objective: To formulate the primal problem and derive the dual problem if there are multiple classes. -->